# ***Face Mask Detection: VGG19***

In [ ]:
import os
import numpy as np
import pandas as pd
import sklearn

# Deep Learning libraries
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image_dataset_from_directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        break

In [ ]:
base_dir = "../input/face-mask-12k-images-dataset/Face Mask Dataset"
train_dir = os.path.join(base_dir, "Train")
validation_dir = os.path.join(base_dir, 'Validation')
test_dir = os.path.join(base_dir, "Test")

## ***Image Augmentation: Using ImageDataGenerator***

In [ ]:
# For Image Augmentation: Generate instance of ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   horizontal_flip=True,
                                   zoom_range=0.3,
                                   fill_mode="nearest")

test_datagen = ImageDataGenerator(rescale=1./255)

## ***Make train, test & validation sets***

In [ ]:
# Creating the dataset of augmented images: Images get augmented only during training
# This will be passed to the model.fit() during training for using the augmented images

train_dataset = train_datagen.flow_from_directory(train_dir,
                                                  target_size=(160,160),
                                                  batch_size=128,
                                                  class_mode="binary")

validation_dataset = test_datagen.flow_from_directory(validation_dir,
                                                      target_size=(160, 160),
                                                      batch_size=128,
                                                      class_mode="binary")
test_dataset = test_datagen.flow_from_directory(test_dir,
                                                target_size=(160, 160),
                                                batch_size=128,
                                                class_mode="binary")

## ***Transfer Learning: Using VGG19 model***

In [ ]:
image_size = (160,160)

# Load the model that we want --- VGG19
base_model = keras.applications.VGG19(include_top=False,
                                            weights='imagenet',
                                            input_shape=image_size+(3,))
base_model.trainable = False

## ***Adding a top to the base VGG19 model***

In [ ]:
# Unfreeze all the layers
base_model.trainable = True

# Freeze the bottom layers
for layer in base_model.layers[:18]:
  layer.trainable = False



In [ ]:
model = keras.Sequential([
      base_model,
      keras.layers.Flatten(),
      keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.summary()

## Compile & run the model

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(),
              loss='binary_crossentropy',
              metrics=['binary_accuracy'])
model.summary()
# learning_rate=0.001,decay=1e-6

In [ ]:

total_epochs = 20
history_fine = model.fit(train_dataset,
                         batch_size=128,
                         epochs=total_epochs,
                         validation_data=validation_dataset)

## ***Evaluate the model on test set(Unseen data)***

In [ ]:
loss,accuracy = model.evaluate(test_dataset)
print("Loss: ", loss)
print("Accuracy: ", accuracy*100)

 # **The model shows a very low loss & 99.697% accuracy**

In [ ]:
# Save the model in h5 format

model.save("VGG19-facemask.h5")


# **Testing on images**

In [ ]:
import cv2

img = cv2.imread(r'../input/face-mask-12k-images-dataset/Face Mask Dataset/Test/WithMask/225.png')
img = cv2.resize(img,(160,160))
img = np.reshape(img,[1,160,160,3])

In [ ]:
from IPython.display import Image
Image(filename=r'../input/face-mask-12k-images-dataset/Face Mask Dataset/Test/WithMask/225.png')

In [ ]:
prediction = model.predict(img)
prediction

In [ ]:
img = cv2.imread(r'../input/face-mask-12k-images-dataset/Face Mask Dataset/Test/WithoutMask/1639.png')
img = cv2.resize(img,(160,160))
img = np.reshape(img,[1,160,160,3])

Image(filename=r'../input/face-mask-12k-images-dataset/Face Mask Dataset/Test/WithoutMask/1639.png')



In [ ]:
prediction = model.predict(img)
prediction

# **Function to get output for a given input face image.**

In [ ]:
def predict_funct(img):
    
    img = cv2.resize(img,(160,160))
    img = np.reshape(img,[1,160,160,3])
    prediction = model.predict(img)
    if prediction == [[1.]]:
          print("Prediction : Without Mask") 
    else: 
          print('Prediction : With Mask')                 

In [ ]:
img = cv2.imread(r'../input/face-mask-12k-images-dataset/Face Mask Dataset/Test/WithoutMask/1639.png')
predict_funct(img)